# Run queries vs AWS Lambda functions
---

In [ ]:
from PIL import Image
import io
import requests
import base64
import time
import json
import numpy as np

import multiprocessing as mp

In [ ]:
with open('test-dog.jpg', 'rb') as f:
    b64_im = base64.b64encode(f.read())

In [ ]:
img = Image.open(io.BytesIO(base64.b64decode(b64_im)))

In [ ]:
# %%timeit

now=time.time()
url = 'https://53f8w4fcua.execute-api.us-east-1.amazonaws.com/Prod/invocations'
payload = {'data': b64_im.decode('utf-8')}
    
headers = {'content-type': 'application/json'}

r = requests.post(url, json=payload, headers=headers)
print('proc time: {} seconds and response is: {}'.format(time.time()-now, r))

In [ ]:
response = r.json()
print(response)

In [ ]:
bb = json.loads(response['json'])['boxes'][0]
area = (bb[0], bb[1], bb[2], bb[3])
cropped_img = img.crop(area)
cropped_img

In [ ]:
area = (response['xmin'], 
        response['ymin'], 
        response['xmax'], 
        response['ymax'])
cropped_img = img.crop(area)
cropped_img

## Parallel requests

In [ ]:
list_of_images = [b64_im for x in range(1000)]

In [ ]:
def query_image(image):
    payload = {'data': image.decode('utf-8')}
    headers = {'content-type': 'application/json'}

    return requests.post(url, json=payload, headers=headers)

In [ ]:
now = time.time()
pool = mp.Pool(mp.cpu_count())
results = pool.starmap(query_image, [(image,) for image in list_of_images])
pool.close()
print('total proc time: {} milliseconds'.format(time.time()-now, r))